In [1]:
import ray
import torch
from matplotlib import pyplot as plt
from timeit import default_timer as timer

import CsvConverter as conv
import pandas as pd
from transformers import pipeline
import seaborn as sns

In [2]:
#import os
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()

ZSC + NLI to match user stories to components 

In [3]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

2.2.2+cu121
True
1
cuda:0


In [4]:
name = "deberta_remain_reduce_8192"
url = 'D:/Thesis/MySQL_GroundTruth/ClassifierOutput/' + name

bart = "facebook/bart-large-mnli"
deberta_base = "MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli"
deberta_large = "MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli"
sileod = 'sileod/deberta-v3-base-tasksource-nli'

In [5]:
def get_sequence(url, columns, column):
    sequence = pd.read_csv(url, delimiter=None, sep=None)
    sequence.columns = columns
    return sequence[column].tolist()


def get_labels(url, delimiter):
    df = pd.read_csv(url, header=None, encoding='ISO-8859-1', delimiter=delimiter)
    return df[0].tolist()

In [6]:
@ray.remote(num_gpus=1, max_calls=1)
def classification(name):
    print(torch.cuda.is_available())
    print(torch.cuda.device_count())
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)
    # loads ZSC from huggingface
    classifier = pipeline('zero-shot-classification', device=device, model=deberta_base, batch_size=8)

    #seq = get_sequence('assets/UserStoriesWithComponents_cleaned_filtered_no_title.csv', ['Type', 'Component_Names', 'TitleAndDescription'], 'TitleAndDescription')
    seq = get_sequence('assets/UserStoriesWithComponents_remain_reduce_lower8000.csv', ['ID', 'Description', 'Type', 'Component_Names'], 'Description')
    labels = get_labels('assets/components/remain_reduce_comps.csv', ';')
    # do the classification
    start = timer()
    results = classifier(seq, labels, multi_label=True)
    end = timer()
    print(end-start)
    print("Done")
    return results, seq

In [7]:
results,seq = ray.get(classification.remote(name))
#results, seq = classification(name)

2024-05-14 09:40:05,755	INFO worker.py:1749 -- Started a local Ray instance.


(classification pid=10396) True
(classification pid=10396) 1
(classification pid=10396) cuda:0


(classification pid=10396) C:\Users\Coco\AppData\Local\Temp\ipykernel_15480\4294111192.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
(classification pid=10396) Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


KeyboardInterrupt: 

In [ ]:
with open('ClassifierOutput/' + name + '.txt', 'w', encoding='utf-8') as f:
    for story, result in zip(seq, results):
        f.write(f"Story: {story}\n")
        for label, score in zip(result['labels'], result['scores']):
            f.write(f"- {label}: {score:.2f}\n")

In [ ]:
#csv = conv.CsvConverter(url + '.txt', url + '.csv', 'Story')
#csv.convert()

In [ ]:
import csv
read_url = url + '.txt'
save_url = url + '.csv'
keyword = 'Story'
with open(read_url, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# dictonary, data is going to be saved to
data = {}
skill = None
for line in lines:
    if line.strip():
        key, value = line.replace("\n", "").split(': ', 1)
        #print(key, value)
        if key.startswith(keyword):
            skill = value
            data[skill] = {}
        else:
            component, weight = key.replace("- ", "", 1), float(value)
            data[skill][component] = weight

# convert to a better format
output_data = {}
for skill, components in data.items():
    for component, weight in components.items():
        if component not in output_data:
            output_data[component] = {}
        output_data[component][skill] = weight

# saving the file
with open(save_url, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    cleaned_headers = [header.strip().strip('"') for header in list(data.keys())]
    cleaned_headers.insert(0, 'Skills')
    writer.writerow(cleaned_headers)
    for component, values in output_data.items():
        row = [component] + [values.get(skill, '') for skill in data.keys()]
        writer.writerow(row)

# Read the CSV file into a DataFrame and sort it
df = pd.read_csv(save_url, encoding='ISO-8859-1')
df_sorted = df.sort_values(by='Skills')
df_sorted.to_csv(save_url, index=False)

In [ ]:
"""name = "bart_remain_reduce"
url = 'D:/Thesis/MySQL_GroundTruth/ClassifierOutput/' + name
def create(title, name, url):
    # Load data from CSV
    data = pd.read_csv(url + '.csv', header=0, index_col=0, encoding='ISO-8859-1', delimiter=None, sep=None)  # Update with your file path
    plt.figure(figsize=(40, 40))
    heatmap = sns.heatmap(data, cmap='Blues', annot=True)
    plt.title(title)
    plt.savefig('heatmap/heatmap_' + name + '.png')


create(name, name, url)"""